In [1]:
"""数据预处理"""

'数据预处理'

In [2]:
#载入需要的库
#import tensorflow as tf
import numpy as np
import os
import random
import cv2

数据准备
读取train和test两个文件夹中的图片，并转化为模型能够加载的格式，并将train分割成train_data和val_data。最终转换的数据为train_X,train_y,val_X,val_y,test_X。

In [2]:
def read_file(dir_path,random_state=True):
    """获取图片路径及其标签
    ：param dir_path:a string,图片所在目录
    ：param random_state:bool,是否乱序
    ：return：
    """
    img_list = []
    label_list = []
    
    cat_num = 0
    dog_num = 0
    for filename in os.listdir(dir_path):
        filename = dir_path + '/'+ filename
        label = filename.split('.')[0]
        
        if os.path.isfile(filename):
            img_list.append(filename)
        else:
            raise ValueError("文件夹中有非文件项")
        
        if label == 'cat':
            label_list.append(0)
            cat_num += 1
        else:
            label_list.append(1)
            dog_num += 1
        print('数据集中有{}只猫，{}只狗'.format(cat_num,dog_num))
        
        #将数据转化为array
        img_list = np.asarray(img_list)#路径
        label_list = np.asarray(label_list)
        
        #打乱文件顺序
        if random_state:
            index_random = np.array(len(img_list))
            np.random.shuffle(index_random)
            img_list = img_list[index_random]
            label_list = label_list[index_random]
        
        return img_list, label_list

In [ ]:
dir_path = './dogs vs. cats/train'
img_list,label_list = read_file(dir_path)

In [3]:
def random_sample(images=img_list,images_category=label_list):
    print('随机选取一张图片：')
    #从 images 和 images_name 随机读取一个图像文件路径以及该图像的类别
    index_random = random.randint(0,len(images))
    im_file,im_name=images[index_random],images_category[index_random]
    #使用cv2.imread 读取图像文件
    img = cv2.imread(im_file)
    #使用plt.imshow 和plt.show()显示图像
    plt.imshow()
    print(im_name)
    print(img.shape)
    return im_file,im_name

NameError: name 'img_list' is not defined

In [ ]:
#定义文件批次函数
#因为一次性将所有25000张图片载入内存不现实也不必要，所以将图片分成不同批次进行训练。
#这里传入的image和label参数就是函数get_files()返回的image_list和label_list，是python中的list类型，
#所以需要将其转为TensorFlow可以识别的tensor格式。 
def get_batch(train_list,img_size,batch_size,capacity,random_state=True):
    """
    获取训练批次
    :param train_list:2-D list,[img_list,label_list]
    :param img_size:a int,训练批次大小
    :param batch_size:a int，每个批次包含的样本数量
    :param capacity:a int,队列容量
    :param random_state:bool,是否乱序
    """
    input_queue = tf.train.slice_input_producer(train_list,shuffle=False)
    
    #从路径中读取图片
    image_train = tf.read_file(input_queue[0])
    image_train = tf.image.decode_jpeg(image_train,channel=3)#将.jpg转化为.JPEG
    image_train = tf.image.resize_images(image_train,[img_size,img_size])
    image_train = tf.cast(image_train,tf.float32)/255 #转化数据类型并归一化
    
    #图片标签
    label_list = input_queue[1]
    
    #获取批次
    if random_state:
        image_train_batch,label_train_batch = tf.train.shuffle([image_train,label_train],
                                                              batch_size=batch_size,
                                                              capacity = capacity,
                                                              min_after_dequeue=100,
                                                              num_threads=2)
    else:
        image_train_batch, label_train_batch = tf.train.batch([image_train, label_train],
                                                              batch_size=1,
                                                              capacity=capacity,
                                                              num_threads=1)
    return image_train_batch, label_train_batch

In [ ]:
"""def load_dataset(data_dir = "./dog vs.cats/train/"):
    data_x = []
    data_y = label_list
    
    for img in img_list:
        img = cv2.imread(img)
        data_x.append(img)
    return data_x,data_y
train_x,train_y = load_dataset("./dog vs.cats/train/")
test_x,test_y = load_dataset("./dog vs.cats/test/")

train_X = np.asarray(train_x)[:10000]
val_X = np.asarray(train_x)[10000:]
train_Y = np.asarray(train_y)[:10000]
val_Y = np.asarray(train_y)[10000:]
test_X = np.asarray(test_x)"""